# fse - Tutorial

Welcome to fse - fast sentence embeddings. The library is intended to compute sentence embeddings as fast as possible. 
It offers a simple and easy to understand syntax for you to use in your own projects. Before we start with any model, lets have a look at the input types.
All fse models require an iterable/generator which produces a tuple. The tuple has two fields: words and index. The index is required for the multi-thread processing, as sentences might not be processed sequentially. The index dictates, which row of the corresponding sentence vector matrix the sentence belongs to.

## Input handling

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
s = (["Hello", "world"], 0)
print(s[0])
print(s[1])

['Hello', 'world']
0


The words of the tuple will always consist of a list of strings. Otherwise the train method will raise an Error. However, most input data is available as a list of strings.

In order to deal with this common input format, fse provides the IndexedList and some variants, which handel all required data operations for you. You can provide multiple lists (or sets) which will all be merged into a single list. This eases work if you have to work with the STS datasets.

The multiple types of indexed lists. Let's go through them one by one:
- IndexedList: for already pre-splitted sentences
- **C**IndexedList: for already pre-splitted sentences with a custom index for each sentence
- SplitIndexedList: for sentences which have not been splitted. Will split the strings
- Split**C**IndexedList: for sentences which have not been splitted and with a custom index for each sentence
- **C**SplitIndexedList: for sentences which have not been splitted. Will split the strings. You can provide a custom split function
- **C**Split*C*IndexedList: for sentences where you want to provide a custom index and a custom split function.

*Note*: These are ordered by speed. Meaning, that IndexedList is the fastest, while **C**Split*C*IndexedList is the slowest variant.

In [3]:
from fse import SplitIndexedList

sentences_a = ["Hello there", "how are you?"]
sentences_b = ["today is a good day", "Lorem ipsum"]

s = SplitIndexedList(sentences_a, sentences_b)
print(len(s))
s[0]

2019-09-11 12:06:10,972 : MainThread : INFO : 'pattern' package not found; tag filters are not available for English


4


(['Hello', 'there'], 0)

To save memory, we do not convert the original lists inplace. The conversion will only take place once you call the getitem method. To access the original data, call:

In [4]:
s.items

['Hello there', 'how are you?', 'today is a good day', 'Lorem ipsum']

If the data is already preprocessed as a list of lists you can just use the IndexedList

In [5]:
from fse import IndexedList

sentences_splitted = ["Hello there".split(), "how are you?".split()]
s = IndexedList(sentences_splitted)
print(len(s))
s[0]

2


(['Hello', 'there'], 0)

In case you want to provide your own splitting function, you can pass a callable to the **C**SplitIndexedList class.

In [6]:
from fse import CSplitIndexedList

def split_func(string):
    return string.lower().split()

s = CSplitIndexedList(sentences_a, custom_split=split_func)
print(len(s))
s[0]

2


(['hello', 'there'], 0)

If you want to stream a file from disk (where each line corresponds to a sentence) you can use the IndexedLineDocument.

In [7]:
from fse import IndexedLineDocument
doc = IndexedLineDocument("../fse/test/test_data/test_sentences.txt")

In [8]:
i = 0
for s in doc:
    print(f"{s[1]}\t{s[0]}")
    i += 1
    if i == 4:
        break

0	['Good', 'stuff', 'i', 'just', 'wish', 'it', 'lasted', 'longer']
1	['Hp', 'makes', 'qualilty', 'stuff']
2	['I', 'like', 'it']
3	['Try', 'it', 'you', 'will', 'like', 'it']


If you are later working with the similarity of sentences, the IndexedLineDocument provides you the option to access each line by its corresponding index. This helps you in determining the similarity of sentences, as the most_similar method would otherwise just return indices.

In [9]:
doc[20]

'I feel like i just got screwed'

# Training a model / Performing inference

Training a fse model is simple. You only need a pre-trained word embedding model which you use during the initializiation of the fse model you want to use.

In [10]:
import gensim.downloader as api
data = api.load("quora-duplicate-questions")
glove = api.load("glove-wiki-gigaword-100")

2019-09-11 12:06:12,050 : MainThread : INFO : loading projection weights from /Users/oliverborchers/gensim-data/glove-wiki-gigaword-100/glove-wiki-gigaword-100.gz
2019-09-11 12:06:56,394 : MainThread : INFO : loaded (400000, 100) matrix from /Users/oliverborchers/gensim-data/glove-wiki-gigaword-100/glove-wiki-gigaword-100.gz


In [11]:
sentences = []
for d in data:
    # Let's blow up the data a bit by replicating each sentence.
    for i in range(8):
        sentences.append(d["question1"].split())
        sentences.append(d["question2"].split())
s = IndexedList(sentences)
print(len(s))

/Users/oliverborchers/anaconda3/envs/fsedev/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


6468640


So we have about 6468640 sentences we want to compute the embeddings for. If you import the FAST_VERSION variable as follows you can ensure, that the compiliation of the cython routines worked correctly:

In [12]:
from fse.models.average import FAST_VERSION, MAX_WORDS_IN_BATCH
print(MAX_WORDS_IN_BATCH)
print(FAST_VERSION)
# 1 -> The fast version works

10000
1


In [13]:
from fse.models import SIF
model = SIF(glove, workers=2)

In [14]:
model.train(s)

2019-09-11 12:07:19,386 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-09-11 12:07:24,388 : MainThread : INFO : SCANNING : finished 3902109 sentences with 43127298 words
2019-09-11 12:07:27,459 : MainThread : WARNING : found 16 empty sentences
2019-09-11 12:07:27,459 : MainThread : INFO : finished scanning 6468640 sentences with an average length of 11 and 71556728 total words
2019-09-11 12:07:27,588 : MainThread : INFO : estimated memory for 6468640 sentences with 100 dimensions and 400000 vocabulary: 2621 MB (2 GB)
2019-09-11 12:07:27,589 : MainThread : INFO : initializing sentence vectors for 6468640 sentences
2019-09-11 12:07:44,587 : MainThread : INFO : pre-computing SIF weights for 400000 words
2019-09-11 12:07:44,895 : MainThread : INFO : begin training
2019-09-11 12:07:49,906 : MainThread : INFO : PROGRESS : finished 32.32% with 2090654 sentences and 15883331 words, 418130 sentences/s
2019-09-11 12:07:54,906 : MainThread : INFO : PROGRESS : fini

(6468624, 49255184)

The models training speed revolves at around 400,000 - 500,000 sentences / seconds. That means we finish the task in about 10 seconds. This is **heavily dependent** on the input processing. If you train ram-to-ram it is naturally faster than any ram-to-disk or disk-to-disk varianty. Similarly, the speed depends on the workers.

Once the sif model is trained, you can perform additional inferences for unknown sentences. This two step process for new data is required, as computing the principal components for models like SIF and uSIF will require a fair amount of sentences. If you want the vector for a single sentence (which is out of the training vocab), just use:

In [15]:
tmp = ("Hello my friends".split(), 0)
model.infer([tmp])

2019-09-11 12:08:25,866 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-09-11 12:08:25,868 : MainThread : INFO : finished scanning 1 sentences with an average length of 3 and 3 total words
2019-09-11 12:08:25,874 : MainThread : INFO : removing 1 principal components took 0s


array([[ 0.381964  , -0.19883852,  0.05714178, -0.26080537, -0.39127582,
         0.56997186, -0.08829714,  0.04303323,  0.02415925, -0.18261646,
         0.12377759,  0.16709256,  0.22155528,  0.19370778, -0.263913  ,
        -0.28011692, -0.26631504, -0.04989926, -0.28030562,  0.35429248,
        -0.33491367,  0.4502715 , -0.3593142 , -0.15993649,  0.19240798,
         0.50069076, -0.45003757, -0.25315157,  0.7854811 ,  0.1651807 ,
         0.4882131 ,  0.38489464,  0.71546996,  0.14502618, -0.2963435 ,
         0.2826286 , -0.11362529,  0.04488423,  0.41105896, -0.24948171,
         0.16784175,  0.18146743,  0.56374484, -0.16483444, -0.34314325,
        -0.11086494, -0.71529436,  0.37857842,  0.549762  , -0.21101838,
        -0.15454581, -0.07437952, -0.0044148 , -0.0836513 ,  0.20791832,
         0.01463795,  0.27646336,  0.39419648, -0.30494606, -0.09648022,
         0.47651437,  0.87657607, -0.01992241, -0.23987049, -0.14416814,
         0.04672481,  0.27598107,  0.09775755,  0.1

## Querying the model

In order to query the model or perform similarity computations we can just access the model.sv (sentence vectors) object and use its method. To get a vector for an index, just call

In [16]:
model.sv[0]

array([ 0.10094467, -0.17338434, -0.06120607,  0.34027657,  0.22177696,
       -0.24794774, -0.11074719, -0.14664021, -0.05048759, -0.00544697,
       -0.1003525 , -0.00855931, -0.12817517, -0.23004776, -0.1799999 ,
        0.07368518, -0.03067313, -0.05431814,  0.08335009, -0.10299531,
        0.068488  , -0.00242307, -0.04005107,  0.29239285, -0.08163965,
       -0.0883536 ,  0.1218466 ,  0.1400187 , -0.07441948,  0.14240132,
       -0.15721212, -0.02932566, -0.06760302, -0.07329973, -0.03554938,
        0.08278871,  0.19171   , -0.05927336, -0.14163393, -0.1375518 ,
       -0.02026191, -0.04314214, -0.05729384,  0.0653533 , -0.06144975,
        0.06533504, -0.09953416,  0.04081041, -0.03480543, -0.19186741,
       -0.21262895,  0.09202442,  0.10283872,  0.12685192, -0.16222198,
       -0.21388721, -0.24895632, -0.16497527,  0.29118574, -0.19552866,
       -0.0310809 , -0.3014589 , -0.00456905,  0.11545968, -0.11587107,
        0.06897421, -0.25824726,  0.0791491 ,  0.16394164,  0.06

To compute the similarity or distance between two sentence from the training set you can call:

In [17]:
print(model.sv.similarity(0,1).round(3))
print(model.sv.distance(0,1).round(3))

0.929
0.071


We can further call for the most similar sentences given an index. For example, we want to know the most similar sentences for sentence index 100:

In [18]:
print(s[100])

(['Should', 'I', 'buy', 'tiago?'], 100)


In [19]:
model.sv.most_similar(100)
# Division by zero can happen if you encounter empy sentences

2019-09-11 12:08:25,930 : MainThread : INFO : precomputing L2-norms of sentence vectors


[(3949083, 0.9999999403953552),
 (897678, 0.9999999403953552),
 (4229890, 0.9999999403953552),
 (3949079, 0.9999999403953552),
 (3949081, 0.9999999403953552),
 (2934317, 0.9999999403953552),
 (4093542, 0.9999999403953552),
 (3949075, 0.9999999403953552),
 (4229889, 0.9999999403953552),
 (2934319, 0.9999999403953552)]

However, the preceding function will only supply the indices of the most similar sentences. You can circumvent this problem by passing an indexable function to the most_similar call:

In [20]:
model.sv.most_similar(100, indexable=s.items)

[(['Should', 'I', 'buy', 'Asus', 'Zenfone', '5?'],
  3949083,
  0.9999999403953552),
 (['Why', "doesn't", 'Google', 'buy', 'Quora?'], 897678, 0.9999999403953552),
 (['Should', 'Google', 'buy', 'Quora?'], 4229890, 0.9999999403953552),
 (['Should', 'I', 'buy', 'Asus', 'Zenfone', '5?'],
  3949079,
  0.9999999403953552),
 (['Should', 'I', 'buy', 'Asus', 'Zenfone', '5?'],
  3949081,
  0.9999999403953552),
 (['Why', "didn't", 'Facebook', 'buy', 'Twitter?'],
  2934317,
  0.9999999403953552),
 (['Should', 'I', 'buy', 'Xiaomi', 'Redmi', 'Note', '3?', 'Why?'],
  4093542,
  0.9999999403953552),
 (['Should', 'I', 'buy', 'Asus', 'Zenfone', '5?'],
  3949075,
  0.9999999403953552),
 (['Will', 'Google', 'buy', 'Quora?'], 4229889, 0.9999999403953552),
 (['Why', "didn't", 'Facebook', 'buy', 'Twitter?'],
  2934319,
  0.9999999403953552)]

There we go. This is a lot more understandable than the initial list of indices.

To search for sentences, which are similar to a given word vector, you can call:

In [21]:
model.sv.similar_by_word("easy", wv=glove, indexable=s.items)

[(['How', 'do', 'I', 'make', 'easy', 'money?'], 6383261, 0.5409939289093018),
 (['How', 'do', 'I', 'make', 'easy', 'money?'], 842112, 0.5409939289093018),
 (['How', 'do', 'I', 'make', 'easy', 'money?'], 6383255, 0.5409939289093018),
 (['How', 'do', 'I', 'make', 'easy', 'money?'], 6383257, 0.5409939289093018),
 (['How', 'do', 'I', 'make', 'easy', 'money?'], 6383259, 0.5409939289093018),
 (['How', 'do', 'I', 'make', 'easy', 'money?'], 4405391, 0.5409939289093018),
 (['How', 'do', 'I', 'make', 'easy', 'money?'], 6383263, 0.5409939289093018),
 (['How', 'do', 'I', 'make', 'easy', 'money?'], 842116, 0.5409939289093018),
 (['How', 'do', 'I', 'make', 'easy', 'money?'], 842114, 0.5409939289093018),
 (['How', 'do', 'I', 'make', 'easy', 'money?'], 6383251, 0.5409939289093018)]

Furthermore, you can query for unknown (or new) sentences by calling:

In [22]:
model.sv.similar_by_sentence("Is this really easy to learn".split(), model=model, indexable=s.items)

2019-09-11 12:08:35,263 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-09-11 12:08:35,266 : MainThread : INFO : finished scanning 1 sentences with an average length of 6 and 6 total words
2019-09-11 12:08:35,270 : MainThread : INFO : removing 1 principal components took 0s


[(['Is',
   'it',
   'easy',
   'to',
   'learn',
   'Hebrew',
   'if',
   'you',
   'learn',
   'Arabic?'],
  4666689,
  0.9029382467269897),
 (['Is',
   'it',
   'easy',
   'to',
   'learn',
   'Arabic',
   'if',
   'you',
   'learn',
   'Hebrew',
   'first?'],
  4666700,
  0.9029382467269897),
 (['Is',
   'it',
   'easy',
   'to',
   'learn',
   'Arabic',
   'if',
   'you',
   'learn',
   'Hebrew',
   'first?'],
  4666698,
  0.9029382467269897),
 (['Is',
   'it',
   'easy',
   'to',
   'learn',
   'Arabic',
   'if',
   'you',
   'learn',
   'Hebrew',
   'first?'],
  4666690,
  0.9029382467269897),
 (['Is',
   'it',
   'easy',
   'to',
   'learn',
   'Hebrew',
   'if',
   'you',
   'learn',
   'Arabic?'],
  4666691,
  0.9029382467269897),
 (['Is',
   'it',
   'easy',
   'to',
   'learn',
   'Hebrew',
   'if',
   'you',
   'learn',
   'Arabic?'],
  4666699,
  0.9029382467269897),
 (['Is',
   'it',
   'easy',
   'to',
   'learn',
   'Arabic',
   'if',
   'you',
   'learn',
   'Hebrew',

Feel free to browse through the library and get to know the functions a little better!